find . -type f -name "DataCite_Public_Data_File_2024.tar" -print0 | \
xargs -0 -I{} tar -O -xf {} --wildcards 'dois/*/*.jsonl.gz' | \
gunzip -c | \
jq -c '.attributes | {doi, subjects, geoLocations, dates}' | \
gzip > output.jsonl.gz

In [ ]:
from collections import Counter
import gzip
import json

scheme_counts = Counter()
uniq_scheme_counts = Counter()
geo_count = 0
date_coverage = 0

with gzip.open('output.jsonl.gz', 'rt', encoding='utf-8') as f:
    for line_num, line in enumerate(f, 1):
        obj = json.loads(line)
        
        if 'geoLocations' in obj and not obj['geoLocations'] == []:
            geo_count += 1

        for date in obj.get('dates', []):
            if isinstance(date, dict) and date.get('dateType') == 'Coverage':
                date_coverage += 1

        scheme_uris_seen = set()
        for subject in obj.get('subjects', []):
            if isinstance(subject, dict):
                scheme_uri = subject.get('schemeUri')
                if scheme_uri:
                    scheme_uris_seen.add(scheme_uri)
                    scheme_counts[scheme_uri] += 1
        for uri in scheme_uris_seen:
            uniq_scheme_counts[uri] += 1

print(dict(scheme_counts))
print(dict(uniq_scheme_counts))
print(f"Dates of the type Coverage: {date_coverage}")
print(f"Records with geoLocations: {geo_count}")
